# Baseline Model
This is a first shot at generation blogposts. This model is meant to be set up easy and quick and produce some result, not necessaryly usable text. I will try it GPT2 first, maybe also something else. 

## GPT2 only, no fine-tuning
htis one is really easy to set up, we basically only need the pipeline. The task at hand is text generation, and our model will be gpt2. 

In [ ]:
pip install transformers

In [5]:
from transformers import pipeline
blogPoster = pipeline(task='text-generation', model='gpt2')

We can now call this with no arguments or an empty string as a starting point and it will give some text. However, it won't per default write about NFTs but about a wide range of topics from the texts, it has been trained on. 

To get an idea, we can generate a few of those. 

In [6]:
blogPoster("", num_return_sequences=5)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'Honeycomb County and St. Louis County have proposed a ballot measure next month seeking to ban the use of the term "superdelegates." The measure would force counties and other parties who run for office to identify themselves as superdelegates by'},
 {'generated_text': 'Mullet\n\nMullet is a powerful tool for generating complex visual objects that are easily created on the fly from HTML document elements.\n\nBy creating large areas of the page, or parts of the page that require large processing cycles'},
 {'generated_text': '"I was not a big fan of it. It\'s too far removed from the way you like us to be and I\'m looking at the game and we\'re happy to see what\'s going to come out."'},
 {'generated_text': '"The way we saw it at least at times this last year, we wanted to say, "OK, here we go, now move fast and be agile,\'" and there\'s very little that\'s happened to that." As the team and business'},
 {'generated_text': "\nWhat if you just love what you see? It's

We can push it a bit in the right direction by giving it something to start with e.g. "The next NFT-based..."

In [7]:
blogPoster("The next NFT-based", num_return_sequences=5)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'The next NFT-based model to get a closer look at how a human brain functions, it could be that the human brain works with a specialized neural network that is known as a "feedback layer."\n\n"We are going to look'},
 {'generated_text': "The next NFT-based approach is simply using an embedded GPU to play fast and provide a bit more CPU.\n\nIt's worth mentioning that this approach was designed for the ARMv8 architecture, and since ARM is going to be using embedded"},
 {'generated_text': 'The next NFT-based data source — what a huge job it is! — is the National Longitudinal Study of Youth, which uses more than one million young people to gather historical data on their own. The data is published by Statistics Canada,'},
 {'generated_text': 'The next NFT-based program (where the software becomes the only thing that drives development to be as good as the software itself), a new NFT-based system for AI-based simulation of real-world scenarios will give the world an impr

Ok, so that was quite bad, as was to be expected, probably. I think we will have to do some fine-tuning.

## Fine-tune GPT2 with the NFT articles data set 
The idea is to take the gpt2-model and train it a few epochs on the NFT articles data set. This will make it more probable to genereate relevant output without the need for a full training from scratch. So we use transfer learning to get there faster. This approach is called domain adaption. So we want to adapt the gpt2-model to the domain of creating NFT-related articles. 

There are two tutorials to follow here: 
https://huggingface.co/course/chapter7/6?fw=pt
This one is on Training a causal language model from scratch, and the other one https://huggingface.co/course/chapter3/3?fw=tf
is on fine-tuning a pretrained model. Those two can be combined to give the desired result, hopefully. 

In [ ]:
# outline

# import all the nessessary stuff, pretrained models, tokenizers and all

# get and prepare the data set for training

# train for a few epochs

# test and evaluate